In [ ]:
# change working directory to parent dir
# run only once
%cd ..

# Import Statements

In [ ]:
import os
import pandas as pd
import seaborn as sns

from sqlalchemy import select
from sqlalchemy.orm import load_only

from utils.sql import start_sqlsession
from utils.datamodel import Article



# Load Data

In [ ]:
session, engine = start_sqlsession()

with engine.begin() as conn:
    stmt = select(Article).options(load_only(Article.article_md5, Article.source, Article.date_published))
    df = pd.read_sql_query(stmt, conn)

df

In [ ]:
df['year_published'] = df.date_published.dt.year

annual_counts = df.groupby(["source", "year_published"]).count().reset_index()
pretty_table = annual_counts.loc[:, 
                                 ['source', 'year_published', 'id']].rename(
                                     columns={'id': 'n', 'source': 'News Outlet', 'year_published': 'Year'}
                                     ).pivot(
                                         index="News Outlet", columns="Year", values='n'
                                         ).fillna(0).astype(int)

pretty_table

pretty_table.to_csv('01_dataquality/article_descriptives.csv')

In [ ]:
order = list(range(annual_counts.year_published.min(), annual_counts.year_published.max() + 1))

g = sns.FacetGrid(annual_counts, col="source", col_wrap=5)
g.map(sns.barplot, "year_published", "id", order=order)
g.set_xticklabels(order, rotation=45)
g.set_xlabels("Year published")
g.set_ylabels("Number of articles")

In [ ]:
g.savefig('01_dataquality/article_descriptives.png')